自動化流程

合并后的音频已保存到：/content/combined_audio.wav


注意事項：雲端運行請設置如下！
根据错误信息和提供的代码，你可以按照以下步骤进行修改：

打开audio.py文件。
找到_build_mel_basis()函数。
修改librosa.filters.mel函数的调用方式，确保所有参数都作为关键字参数传递。

return librosa.filters.mel(sr=hp.sample_rate, n_fft=hp.n_fft, n_mels=hp.num_mels, fmin=hp.fmin, fmax=hp.fmax)


自動化流程

In [ ]:
!pip install python-docx
!rm -rf /content/sample_data
!mkdir /content/sample_data
!git clone https://github.com/zabique/Wav2Lip
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!cd Wav2Lip && pip install -r requirements.txt
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"
!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from IPython.display import clear_output
clear_output()
print("\nDone")
from docx import Document
! pip install python-pptx
!nvidia-smi
!git clone https://github.com/Kedreamix/Linly-Talker
%cd /content/Linly-Talker/
!pip install -r VITS/requirements_gptsovits.txt
!pip install gradio
!rm -rf Linly-Talker/
!git lfs install
!git clone https://huggingface.co/Kedreamix/Linly-Talker
%cd /content/Linly-Talker/
!mkdir checkpoints
%mv Linly-Talker/checkpoints/* ./checkpoints/
%mv Linly-Talker/GPT_SoVITS/pretrained_models/* ./GPT_SoVITS/pretrained_models/
%mv Linly-Talker/Qwen ./
import sys
print(sys.path)
! pip install -r /content/Linly-Talker/VITS/requirements_xtts.txt


import os
import re
import shutil
from docx import Document
from pydub import AudioSegment
from VITS.GPT_SoVITS import GPT_SoVITS

# 初始化和配置文件夹
source_dir = '/content'
target_dir = '/content/doc_folder'
audio_base_dir = '/content/Voice3'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# 移动文档到目标文件夹
for file in os.listdir(source_dir):
    if file.endswith('.docx') and any(char.isdigit() for char in file):
        src_path = os.path.join(source_dir, file)
        dst_path = os.path.join(target_dir, file)
        shutil.move(src_path, dst_path)
        print(f'文件 {file} 已移动到 {target_dir}')

vits = GPT_SoVITS()
gpt_path = "/content/drive/MyDrive/语音权重/testzhang-e50.ckpt"
sovits_path = "/content/drive/MyDrive/语音权重/testzhang_e8_s128.pth"
vits.load_model(gpt_path, sovits_path)

ref_wav_path = "/content/drive/MyDrive/语音权重/20240329.wav_0000156800_0000395200.wav"
prompt_text = "那我们物联网概论总共分为三个章节，好，第一章首先要和大家介绍的是物联网的简介。"
prompt_language = "中文"

def process_docs(folder_path):
    doc_files = [file for file in os.listdir(folder_path) if file.endswith('.docx')]
    doc_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

    for doc_file in doc_files:
        doc_id = re.search(r'\d+', doc_file).group()
        audio_save_dir = os.path.join(audio_base_dir, doc_id)
        os.makedirs(audio_save_dir, exist_ok=True)

        file_path = os.path.join(folder_path, doc_file)
        process_docx(file_path, audio_save_dir, doc_id)

def process_docx(file_path, audio_save_dir, doc_id):
    doc = Document(file_path)
    sentence_id = 0
    combined = AudioSegment.empty()

    for para in doc.paragraphs:
        sentences = re.split(r'[。？?]', para.text)
        for sentence in sentences:
            if sentence.strip():
                save_path = os.path.join(audio_save_dir, f'{sentence_id}.wav')
                vits.predict(ref_wav_path=ref_wav_path,
                             prompt_text=prompt_text,
                             prompt_language=prompt_language,
                             text=sentence.strip(),
                             text_language="中文",
                             how_to_cut="不切",
                             save_path=save_path)
                sound = AudioSegment.from_file(save_path)
                combined += sound
                sentence_id += 1

    output_path = os.path.join(audio_save_dir, f'combine{doc_id}.wav')
    combined.export(output_path, format='wav')
    print(f'合并后的音频已保存到：{output_path}')

process_docs(target_dir)
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q mediapipe opencv-python pydub
import os
os.environ['PYTHONUTF8'] = '1'
import cv2
import mediapipe as mp
import numpy as np
import os
from glob import glob

# 初始化 Selfie Segmentation
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

# 设定基础视频路径和输出文件夹路径
base_video_path = "/content/basevideo.mp4"
output_folder = "/content/combined1"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 获取所有背景图像路径
background_images = sorted(glob("/content/*.PNG"), key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('幻灯片')[1]))

# 处理每一个背景图像
for idx, background_image_path in enumerate(background_images):
    print(f"Processing with background: {background_image_path}")

    # 设置输出视频路径
    output_video_path = os.path.join(output_folder, f"{idx+1}.mp4")

    # 读取背景图像
    background_original = cv2.imread(background_image_path)
    if background_original is None:
        print("Error: Background image could not be read.")
        continue

    # 读取视频文件
    cap = cv2.VideoCapture(base_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        continue

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 调整背景图像大小
    scale = min(width / background_original.shape[1], height / background_original.shape[0])
    new_width = int(background_original.shape[1] * scale)
    new_height = int(background_original.shape[0] * scale)
    background_resized = cv2.resize(background_original, (new_width, new_height))

    # 创建视频帧相同大小的背景
    background = np.zeros((height, width, 3), dtype=np.uint8)
    start_x = (width - new_width) // 2
    start_y = (height - new_height) // 2
    background[start_y:start_y+new_height, start_x:start_x+new_width] = background_resized

    # 初始化视频写入器
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # 设置人物缩放比例和位置
    scale_factor = 0.56
    person_width = int(width * scale_factor)
    person_height = int(height * scale_factor)
    person_x = width - person_width
    person_y = height - person_height

    # 视频处理
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # 应用自拍分割
            results = selfie_segmentation.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.5

            # 提取人物并调整大小
            person = np.where(condition, frame, np.zeros_like(frame))
            person_small = cv2.resize(person, (person_width, person_height))
            person_mask_small = cv2.resize(condition.astype(np.uint8), (person_width, person_height))

            # 合成人物到背景
            new_background = background.copy()
            for i in range(person_height):
                for j in range(person_width):
                    if person_mask_small[i, j, 0]:
                        new_background[person_y+i, person_x+j] = person_small[i, j]

            out.write(new_background)

    finally:
        cap.release()
        out.release()
        print(f"Video {idx+1} processing complete and saved to {output_video_path}")

# 结束 Selfie Segmentation
selfie_segmentation.close()
print("All processing complete.")
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips
from pydub import AudioSegment

def extend_video_to_audio_length(video_path, audio_path, output_path):
    video_clip = VideoFileClip(video_path)
    audio = AudioSegment.from_file(audio_path)
    audio_duration = len(audio) / 1000.0  # 获取音频长度（秒）

    repeat_count = int(audio_duration // video_clip.duration)  # 计算需要重复视频的次数
    last_clip_duration = audio_duration % video_clip.duration  # 最后一段视频的长度

    clips = [video_clip] * repeat_count
    if last_clip_duration > 0:
        last_clip = video_clip.subclip(0, last_clip_duration)
        clips.append(last_clip)

    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile(output_path, codec='libx264')  # 输出视频文件

def process_videos_and_audios(video_folder, audio_base_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

    for video_file in video_files:
        video_index = video_file.split('.')[0].replace('video', '')  # 从视频文件名中提取编号
        audio_folder = os.path.join(audio_base_folder, video_index)  # 根据编号找到对应的子文件夹

        if os.path.exists(audio_folder):
            audio_files = [f for f in os.listdir(audio_folder) if 'combine' in f and f.endswith('.wav')]
            if audio_files:
                video_path = os.path.join(video_folder, video_file)
                audio_path = os.path.join(audio_folder, audio_files[0])
                output_video_path = os.path.join(output_folder, f'combine{video_index}.mp4')
                extend_video_to_audio_length(video_path, audio_path, output_video_path)
                print(f"Extended video saved to {output_video_path}")
            else:
                print(f"No 'combine' audio files found in folder {audio_folder} for video {video_file}")
        else:
            print(f"Audio folder {audio_folder} does not exist for video {video_file}")

process_videos_and_audios('/content/combined1', '/content/Voice3', '/content/extended')
import shutil

def move_specific_wav_files(source_dir, target_dir):
    # 确保目标目录存在，如果不存在则创建
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # 遍历源目录中的所有子目录
    for subdir, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.wav') and 'combine' in file:
                # 构建源文件和目标文件的完整路径
                source_file = os.path.join(subdir, file)
                target_file = os.path.join(target_dir, file)

                # 移动文件
                shutil.move(source_file, target_file)
                print(f"Moved '{source_file}' to '{target_file}'")

# 指定源目录和目标目录
source_directory = '/content/Voice3'
target_directory = '/content/extended'

# 调用函数
move_specific_wav_files(source_directory, target_directory)

import os
import subprocess
import shutil

os.chdir('/content/Wav2Lip')
def run_inference_on_pairs(face_audio_dir, output_dir, checkpoint_path):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    files = os.listdir(face_audio_dir)
    prefixes = set([os.path.splitext(file)[0] for file in files if file.startswith('combine')])

    for prefix in prefixes:
        face_path = os.path.join(face_audio_dir, f"{prefix}.mp4")
        audio_path = os.path.join(face_audio_dir, f"{prefix}.wav")

        if os.path.exists(face_path) and os.path.exists(audio_path):
            output_path = os.path.join(output_dir, f"{prefix}_result.mp4")

            command = [
                'python', '/content/Wav2Lip/inference.py',
                '--checkpoint_path', checkpoint_path,
                '--face', face_path,
                '--audio', audio_path,
                '--outfile', output_path
            ]
            subprocess.run(command, check=True)
            print(f"Inference completed for {prefix}. Result saved to {output_path}")

face_audio_directory = '/content/extended'
final_output_directory = '/content/final_result'
checkpoint_path = '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'

run_inference_on_pairs(face_audio_directory, final_output_directory, checkpoint_path)
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os

def sort_and_concatenate_videos(directory, output_path):
    # 获取所有的视频文件
    files = [file for file in os.listdir(directory) if file.endswith('.mp4')]
    # 按照文件名中的数字进行排序
    files.sort(key=lambda x: int(x.split('_')[0].replace('combine', '')))

    # 加载视频文件为 VideoFileClip 对象
    clips = [VideoFileClip(os.path.join(directory, file)) for file in files]

    # 使用 concatenate_videoclips 合并视频
    final_clip = concatenate_videoclips(clips, method='compose')

    # 写出最终的视频文件
    final_clip.write_videofile(output_path, codec='libx264')

# 指定目录和输出视频文件的路径
source_directory = '/content/final_result'
final_video_path = '/content/final_combined_video.mp4'

# 调用函数
sort_and_concatenate_videos(source_directory, final_video_path)
from google.colab import files
files.download('/content/final_combined_video.mp4')


Done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 8.8 MB/s eta 0:00:00
Thu Apr 25 07:00:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              10W /  70W |      0MiB / 15360MiB |      0%    

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.0 MB/

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
Some weights of the model checkpoint at GPT_SoVITS/pretrained_models/chinese-hubert-base were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the mo

GPT_SoVITS导入成功
XTTS导入失败，原因： cannot import name 'RootModel' from 'pydantic' (/usr/local/lib/python3.10/dist-packages/pydantic/__init__.cpython-310-x86_64-linux-gnu.so)
使用XTTS语音克隆前需要安装对应的环境，请执行 pip install -r VITS/requirements_xtts.txt
文件 2.docx 已移动到 /content/doc_folder
文件 3.docx 已移动到 /content/doc_folder
文件 1.docx 已移动到 /content/doc_folder
Number of parameter: 77.49M


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


...

MoviePy - Done.
Moviepy - Writing video /content/final_combined_video.mp4



t:  58%|█████▊    | 469/806 [00:38<00:37,  9.10it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/final_result/combine1_result.mp4, 6220800 bytes wanted but 0 bytes read,at frame 468/472, at time 15.60/15.72 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  58%|█████▊    | 470/806 [00:38<00:37,  8.99it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/final_result/combine1_result.mp4, 6220800 bytes wanted but 0 bytes read,at frame 469/472, at time 15.63/15.72 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  59%|█████▊    | 472/806 [00:38<00:34,  9.76it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/mo

Moviepy - Done !
Moviepy - video ready /content/final_combined_video.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>